Phase 1 - Notebook 1 (copied from Databricks, not all code may work correctly in other IDEs).

This notebook queries new FHIR data and converts it to a rectangular format.

**Author:** Nate Bean

**Date created:** 11/23 -- **Last modified:** 11/23

**Purpose:** Use this script to query data from the production MDH Syphilis FHIR API. Refer to the 'Syphilis FHIR API Manual' document for information  

**Schedule:** Daily

In [0]:
from datetime import datetime, timedelta
import pandas as pd
import os

In [0]:
%run /Shared/InfectiousDiseases/Syphilis/FHIR/Production/fhir_functions

In [0]:
raw = "/dbfs/mnt/phmdw/Raw/PublicHealth/InfectiousDiseases/SyphilisProvisional/Daily Files/"
raw_fhir = "/dbfs/mnt/phmdw/Raw/PublicHealth/InfectiousDiseases/SyphilisProvisional/FHIR Files/"

In [ ]:
#Retrieve credentials from Azure Key Vault
id_kv = dbutils.secrets.get(scope = "PH-PrdMDW-dbw-PrdMDW-kv-secret-scope", key = "mdh-fhir-id") 
secret_kv = dbutils.secrets.get(scope = "PH-PrdMDW-dbw-PrdMDW-kv-secret-scope", key = "mdh-fhir-prod-secret")

#Generate API access token
access_token = get_token(keycloak_url = "https://authenticatorx.web.health.state.mn.us", 
                         realm = "mdh-fhir-gateway-realm",
                         client_id = id_kv,
                         client_secret = secret_kv)

In [0]:
#Define records to include by earliest modification date
#mod_date = "2023-01-01 00:00" #on or after; given that some records may be removed, always query all records
#mod_date = datetime.today().strftime('%Y-%m-%d')

#If there is no new data available, the workflow will exit after this function.
#Currently, the API fails if the query returns more than 100mb of data (~700 records)
#query_json = query_fhir(app_host = "https://sti-fhir-api.web.health.state.mn.us", #prod endpoint
#                        suffix = "/fhir/medss/" + mod_date)

In [ ]:
#Define records to include by earliest modification date
#mod_date = "2023-01-01 00:00" #on or after; given that some records may be removed, always query all records
yesterday = datetime.today() - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')
mod_date = f"{yesterday} 00:00"

#If there is no new data available, the workflow will exit after this function.
#Currently, the API fails if the query returns more than 100mb of data (~700 records)
query_json = query_fhir(app_host = "https://sti-fhir-api.web.health.state.mn.us", #prod endpoint
                        suffix = "/fhir/medss/" + mod_date)

In [0]:
fhir_txt = json.dumps(query_json)
today = datetime.today().strftime('%Y-%m-%d')

with open(raw_fhir + 'fhir_txt_' + today + '.txt', "w") as text_file:
    text_file.write(fhir_txt)

In [ ]:
query_json

In [ ]:
query_data = parse_sti_fhir(query_json)

display(query_data)

In [0]:
#data_date = datetime.today().strftime('%Y.%m.%d')
#query_data.to_csv(raw + "fhir_syphilis_data_" + data_date + ".csv", index = False)

In [ ]:
#Get most recent file
files = os.listdir(raw)
most_recent = files[len(files)-1]

old_data = pd.read_csv(raw + most_recent)
old_data = old_data.astype({"id": str})
print(f"{len(old_data)} rows in the original data")
print(f"{len(query_data)} downloaded records")

old_dataf = old_data[(~old_data["id"].isin(query_data.id))]
print(f"{len(old_data) - len(old_dataf)} records already in data")

updated = pd.concat([old_dataf, query_data])
print(f"{len(updated)} records in updated data")

data_date = datetime.today().strftime('%Y.%m.%d')
updated.to_csv(raw + "fhir_syphilis_data_" + data_date + ".csv", index = False)